# Solutions to Cooee, Inc.'s data engineering test

Jessan Rendell G. Belenzo  
December 10, 2021

<br>

## Importing the dependencies

In [118]:
import csv
import io
import sqlite3
import zipfile

<br>

## Loading the data into the database

#### 1. Describe the file format of `data.zip`

The compressed ZIP file contains 8 tables in TBL format: `supplier.tbl`, `region.tbl`, `partsupp.tbl`, `part.tbl`, `orders.tbl`, `nation.tbl`, `lineitem.tbl`, and `customer.tbl`. The TBL files correspond to the 8 entities illustrated in the entity relationship diagram (ERD). Each TBL file comprises rows with vertical bar-separated values. The columns in each file also align with those shown in the ERD.

<br>

#### 2. Design a star schema model into which the data should flow

In [119]:
con = sqlite3.connect('database.db')
cur = con.cursor()

# Region
cur.execute(
    """
    CREATE TABLE Region (
        r_regionkey INTEGER,
        r_name TEXT,
        r_comment TEXT,
        PRIMARY KEY (r_regionkey)
    );
    """
)

# Nation
cur.execute(
    """
    CREATE TABLE Nation (
        n_nationkey INTEGER,
        n_name TEXT,
        n_regionkey INTEGER,
        n_comment TEXT,
        PRIMARY KEY (n_nationkey),
        FOREIGN KEY (n_regionkey) REFERENCES Region (r_regionkey)
    );
    """
)

# Part
cur.execute(
    """
    CREATE TABLE Part (
        p_partkey INTEGER,
        p_name TEXT,
        p_mfgr TEXT,
        p_brand TEXT,
        p_type TEXT,
        p_size INTEGER,
        p_container TEXT,
        p_retailprice NUMERIC,
        p_comment TEXT,
        PRIMARY KEY (p_partkey)
    );
    """
)

# Customer
# Added a field to classify customers into 3 classes according to their account balance:
# - `5K AND ABOVE`
# - `0 AND ABOVE`
# - `LESS THAN 0`
cur.execute(
    """
    CREATE TABLE Customer (
        c_custkey INTEGER,
        c_name TEXT,
        c_address TEXT,
        c_nationkey INTEGER,
        c_phone TEXT,
        c_acctbal NUMERIC,
        c_class TEXT,
        c_mktsegment TEXT,
        c_comment TEXT,
        PRIMARY KEY (c_custkey),
        FOREIGN KEY (c_nationkey) REFERENCES Nation (n_nationkey)
    );
    """
)

# Supplier
cur.execute(
    """
    CREATE TABLE Supplier (
        s_suppkey INTEGER,
        s_name TEXT,
        s_address TEXT,
        s_nationkey INTEGER,
        s_phone TEXT,
        s_acctbal NUMERIC,
        s_comment TEXT,
        PRIMARY KEY (s_suppkey),
        FOREIGN KEY (s_nationkey) REFERENCES Nation (n_nationkey)
    );
    """
)

# Orders
cur.execute(
    """
    CREATE TABLE Orders (
        o_orderkey INTEGER,
        o_custkey INTEGER,
        o_orderstatus TEXT,
        o_totalprice NUMERIC,
        o_orderdate DATE,
        o_orderpriority TEXT,
        o_clerk TEXT,
        o_shippriority INTEGER,
        o_comment TEXT,
        PRIMARY KEY (o_orderkey),
        FOREIGN KEY (o_custkey) REFERENCES Customer (c_custkey)
    );
    """
)

# Partsupp
cur.execute(
    """
    CREATE TABLE Partsupp (
        ps_id INTEGER,
        ps_partkey INTEGER,
        ps_suppkey INTEGER,
        ps_availqty INTEGER,
        ps_supplycost NUMERIC,
        ps_comment TEXT,
        PRIMARY KEY (ps_id),
        FOREIGN KEY (ps_partkey) REFERENCES Part (p_partkey),
        FOREIGN KEY (ps_suppkey) REFERENCES Supplier (s_suppkey)
    );
    """
)

# Lineitem
# Added a field called `l_revenue`, the computed extended amount per line item
cur.execute(
    """
    CREATE TABLE Lineitem (
        l_id INTEGER,
        l_orderkey INTEGER,
        l_ps_id INTEGER,
        l_linenumber INTEGER,
        l_quantity INTEGER,
        l_extendedprice NUMERIC,
        l_discount NUMERIC,
        l_tax NUMERIC,
        l_revenue NUMERIC,
        l_returnflag TEXT,
        l_linestatus TEXT,
        l_shipdate DATE,
        l_commitdate DATE,
        l_receiptdate DATE,
        l_shipinstruct TEXT,
        l_shipmode TEXT,
        l_comment TEXT,
        FOREIGN KEY (l_orderkey) REFERENCES Orders (o_orderkey),
        FOREIGN KEY (l_ps_id) REFERENCES Partsupp (ps_id)
    );
    """
)

<br>

#### 3. Load the data into the star schema

In [120]:
with zipfile.ZipFile('data.zip') as data:

    # Region
    with data.open('region.tbl') as region:
        reader = csv.reader(io.TextIOWrapper(region, 'utf-8'), delimiter='|')
        for row in reader:
            r_regionkey = int(row[0])
            r_name = row[1]
            r_comment = row[2]

            cur.execute(
                "INSERT INTO Region (r_regionkey, r_name, r_comment) VALUES (?, ?, ?);",
                (r_regionkey, r_name, r_comment)
            )
    
    # Nation
    with data.open('nation.tbl') as nation:
        reader = csv.reader(io.TextIOWrapper(nation, 'utf-8'), delimiter='|')
        for row in reader:
            n_nationkey = int(row[0])
            n_name = row[1]
            n_regionkey = int(row[2])
            n_comment = row[3]

            cur.execute(
                "INSERT INTO Nation (n_nationkey, n_name, n_regionkey, n_comment) VALUES (?, ?, ?, ?);",
                (n_nationkey, n_name, n_regionkey, n_comment),
            )

    # Part
    with data.open('part.tbl') as part:
        reader = csv.reader(io.TextIOWrapper(part, 'utf-8'), delimiter='|')
        for row in reader:
            p_partkey = int(row[0])
            p_name = row[1]
            p_mfgr = row[2]
            p_brand = row[3]
            p_type = row[4]
            p_size = int(row[5])
            p_container = row[6]
            p_retailprice = float(row[7])
            p_comment = row[8]

            cur.execute(
                """
                INSERT INTO Part (p_partkey, p_name, p_mfgr, p_brand, p_type, p_size, p_container, p_retailprice, p_comment)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
                """,
                (p_partkey, p_name, p_mfgr, p_brand, p_type, p_size, p_container, p_retailprice, p_comment),
            )
    
    # Customer
    with data.open('customer.tbl') as customer:
        reader = csv.reader(io.TextIOWrapper(customer, 'utf-8'), delimiter='|')
        for row in reader:
            c_custkey = int(row[0])
            c_name = row[1]
            c_address = row[2]
            c_nationkey = int(row[3])
            c_phone = row[4]
            c_acctbal = float(row[5])
            c_class = (
                '5K AND ABOVE' if c_acctbal >= 5_000.00 else
                '0 AND ABOVE' if c_acctbal >= 0.00 else
                'LESS THAN 0'
            )
            c_mktsegment = row[6]
            c_comment = row[7]

            cur.execute(
                """
                INSERT INTO Customer (c_custkey, c_name, c_address, c_nationkey, c_phone, c_acctbal, c_class, c_mktsegment, c_comment)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
                """,
                (c_custkey, c_name, c_address, c_nationkey, c_phone, c_acctbal, c_class, c_mktsegment, c_comment),
            )
    
    # Supplier
    with data.open('supplier.tbl') as supplier:
        reader = csv.reader(io.TextIOWrapper(supplier, 'utf-8'), delimiter='|')
        for row in reader:
            s_suppkey = int(row[0])
            s_name = row[1]
            s_address = row[2]
            s_nationkey = int(row[3])
            s_phone = row[4]
            s_acctbal = float(row[5])
            s_comment = row[6]
            
            cur.execute(
                """
                INSERT INTO Supplier (s_suppkey, s_name, s_address, s_nationkey, s_phone, s_acctbal, s_comment)
                VALUES (?, ?, ?, ?, ?, ?, ?);
                """,
                (s_suppkey, s_name, s_address, s_nationkey, s_phone, s_acctbal, s_comment),
            )

    # Orders
    with data.open('orders.tbl') as orders:
        reader = csv.reader(io.TextIOWrapper(orders, 'utf-8'), delimiter='|')
        for row in reader:
            o_orderkey = int(row[0])
            o_custkey = int(row[1])
            o_orderstatus = row[2]
            o_totalprice = float(row[3])
            o_orderdate = row[4]
            o_orderpriority = row[5]
            o_clerk = row[6]
            o_shippriority = int(row[7])
            o_comment = row[8]

            cur.execute(
                """
                INSERT INTO Orders (o_orderkey, o_custkey, o_orderstatus, o_totalprice, o_orderdate, o_orderpriority, o_clerk, o_shippriority, o_comment)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
                """,
                (o_orderkey, o_custkey, o_orderstatus, o_totalprice, o_orderdate, o_orderpriority, o_clerk, o_shippriority, o_comment)
            )

    # Partsupp
    with data.open('partsupp.tbl') as partsupp:
        reader = csv.reader(io.TextIOWrapper(partsupp, 'utf-8'), delimiter='|')
        for row in reader:
            ps_partkey = int(row[0])
            ps_suppkey = int(row[1])
            ps_availqty = int(row[2])
            ps_supplycost = float(row[3])
            ps_comment = row[4]

            cur.execute(
                """
                INSERT INTO Partsupp (ps_partkey, ps_suppkey, ps_availqty, ps_supplycost, ps_comment)
                VALUES (?, ?, ?, ?, ?);
                """,
                (ps_partkey, ps_suppkey, ps_availqty, ps_supplycost, ps_comment)
            )

    # Lineitem
    with data.open('lineitem.tbl') as lineitem:
        reader = csv.reader(io.TextIOWrapper(lineitem, 'utf-8'), delimiter='|')
        for row in reader:
            l_id = int(row[0])
            l_orderkey = int(row[1])
            l_ps_id = int(row[2])
            l_linenumber = int(row[3])
            l_quantity = int(row[4])
            l_extendedprice = float(row[5])
            l_discount = float(row[6])
            l_tax = float(row[7])
            l_revenue = round((l_extendedprice - (l_extendedprice * l_discount) + (l_extendedprice * l_tax)) * l_quantity, 2)
            l_returnflag = row[8]
            l_linestatus = row[9]
            l_shipdate = row[10]
            l_commitdate = row[11]
            l_receiptdate = row[12]
            l_shipinstruct = row[13]
            l_shipmode = row[14]
            l_comment = row[15]

            cur.execute(
                """
                INSERT INTO Lineitem (l_id, l_orderkey, l_ps_id, l_linenumber, l_quantity, l_extendedprice, l_discount, l_tax, l_revenue, l_returnflag, l_linestatus, l_shipdate, l_commitdate, l_receiptdate, l_shipinstruct, l_shipmode, l_comment)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                """,
                (l_id, l_orderkey, l_ps_id, l_linenumber, l_quantity, l_extendedprice, l_discount, l_tax, l_revenue, l_returnflag, l_linestatus, l_shipdate, l_commitdate, l_receiptdate, l_shipinstruct, l_shipmode, l_comment,)
            )

<br>

## Reporting the results

#### 1. What are the top 5 nations in terms of revenue?

Throughout the report, `Orders.o_totalprice` is assumed to be the revenue column.

In [121]:
# Get the list of all nations
nations = cur.execute("SELECT DISTINCT n_name FROM Nation;").fetchall()
nations = [nation[0] for nation in nations]

# Compute for the total revenue for each nation
revenues = []
for nation in nations:
    revenue = cur.execute(
        """
        SELECT Nation.n_name, ROUND(SUM(Orders.o_totalprice), 2)
        FROM Orders, Customer, Nation
        WHERE Orders.o_custkey = Customer.c_custkey AND
            Customer.c_nationkey = Nation.n_nationkey AND
            Nation.n_name = ?;
        """,
        (nation,)
    ).fetchall()
    revenues.append(revenue[0])

# Sort the nations by total revenue in descending order
revenues.sort(key=lambda x: x[1], reverse=True)

# Get the top 5 nations in terms of revenue
print('Rank\tNation\tRevenue')
for rank, (nation, revenue) in enumerate(revenues[:5]):
    print(f"{rank + 1}\t{nation}\t{revenue:,}")


Rank	Nation	Revenue
1	CANADA	109,618,039.26
2	EGYPT	106,410,120.38
3	IRAN	104,237,947.76
4	BRAZIL	98,202,854.19
5	ALGERIA	97,421,274.73


The top 5 nations in terms of revenue are Canada, Egypt, Iran, Brazil, and Algeria.

<br>

#### 2. From the top 5 nations, what is the most common shipping mode?

In [122]:
# Top 5 nations in terms of revenue
nations = [nation[0] for nation in revenues[:5]]

# Get the list of unique shipping modes
shipping_modes = cur.execute("select distinct l_shipmode from Lineitem;").fetchall()
shipping_modes = [mode[0] for mode in shipping_modes]

# Get the frequence of shipping modes in the top 5 nations
shipping_mode_count = []
for mode in shipping_modes:
    count = cur.execute(
        """
        SELECT Lineitem.l_shipmode, COUNT(Lineitem.l_shipmode)
        FROM Lineitem, Nation
        WHERE Lineitem.l_shipmode = ? AND
            Nation.n_name IN (?, ?, ?, ?, ?);
        """,
        (mode, nations[0], nations[1], nations[2], nations[3], nations[4])
    ).fetchall()
    shipping_mode_count.append(count[0])

# Sort the shipping modes by count in descending order
shipping_mode_count.sort(key=lambda x: x[1], reverse=True)

# Print the results
print('Rank\tShipping Mode\tCount')
for rank, (mode, count) in enumerate(shipping_mode_count):
    print(f"{rank + 1}\t{mode}\t\t{count:,}")



Rank	Shipping Mode	Count
1	TRUCK		43,550
2	MAIL		43,345
3	FOB		43,205
4	REG AIR		43,080
5	RAIL		42,830
6	AIR		42,455
7	SHIP		42,410


The most common shipping mode is truck.

<br>

#### 3. What are the top selling months?

In [123]:
months = [f"{month:02}" for month in range(1, 13)]

revenues = []
for month in months:
    revenue = cur.execute(
        """
        SELECT strftime('%m', o_orderdate) as month, SUM(o_totalprice)
        FROM Orders
        WHERE month = ?;
        """,
        (month,)
    ).fetchall()
    revenues.append(revenue[0])

# Sort the months by revenue in descending order
revenues.sort(key=lambda x: x[1], reverse=True)

# Print the results
print('Rank\tMonth\tRevenue')
for rank, (month, revenue) in enumerate(revenues[:5]):
    print(f"{rank + 1}\t{month}\t{round(revenue, 2):,}")
    

Rank	Month	Revenue
1	05	194,423,025.45
2	03	191,112,208.41
3	01	188,111,820.23
4	07	185,564,589.11
5	04	184,443,340.92


The top selling months are May, March, January, July, and April.

<br>

#### 4.	Who is the top customer in terms of revenue and/or quantity?

##### By Revenue

In [124]:
# Get the list of unique customers
customers = cur.execute("select distinct c_custkey from customer;").fetchall()
customers = [customer[0] for customer in customers]

# Compute for the total revenues for each customer
revenues = []
for customer in customers:
    revenue = cur.execute(
        """
        SELECT Customer.c_custkey, SUM(Orders.o_totalprice)
        FROM Orders, Customer
        WHERE Customer.c_custkey = ? AND
            Customer.c_custkey = Orders.o_custkey;
        """,
        (customer,)
    ).fetchall()
    revenues.append(revenue[0])

# Ignore the NoneType values
revenues = [revenue for revenue in revenues if revenue[0] is not None or revenue[1] is not None]

# Sort the customers by total revenue in descending order
revenues.sort(key=lambda x: x[1], reverse=True)

# Get the top 5 customers in terms of revenue
print("Top 5 customers by revenue")
print('Rank\tCustomer\tRevenue')
for rank, (customer, revenue) in enumerate(revenues[:5]):
    print(f"{rank + 1}\t{customer}\t\t{round(revenue, 2):,}")

Top 5 customers by revenue
Rank	Customer	Revenue
1	1489		5,408,941.28
2	214		4,674,894.73
3	1396		4,644,936.89
4	1246		4,642,942.33
5	73		4,638,819.21


In terms of revenue, the top customer is Customer#1489.

<br>

##### By Quantity

In [125]:
# Compute for the total quantity for each customer
quantities = []
for customer in customers:
    quantity = cur.execute(
        """
        SELECT Customer.c_custkey, SUM(Lineitem.l_quantity)
        FROM Lineitem, Orders, Customer
        WHERE Customer.c_custkey = ? AND
            Customer.c_custkey = Orders.o_custkey AND
            Orders.o_orderkey = Lineitem.l_orderkey;
        """,
        (customer,)
    ).fetchall()
    quantities.append(quantity[0])

# Ignore the NoneType values
quantities = [quantity for quantity in quantities if quantity[0] is not None or quantity[1] is not None]

# Sort the customers by total revenue in descending order
quantities.sort(key=lambda x: x[1], reverse=True)

# Get the top 5 customers in terms of Quantity
print("Top 5 customers by quantity")
print('Rank\tCustomer\tQuantity')
for rank, (customer, quantity) in enumerate(quantities[:5]):
    print(f"{rank + 1}\t{customer}\t\t{quantity:,}")

Top 5 customers by quantity
Rank	Customer	Quantity
1	1027		3,453
2	685		3,399
3	392		2,719
4	649		2,717
5	1196		2,665


In terms of quantity, the top customer is Customer#1027.

<br>

#### 5.	Compare the sales revenue during current period against those during the previous period.

Because 1998 was the latest year in the data, it was assumed to be the current period.

In [126]:
current_period = cur.execute(
    """
    SELECT strftime('%Y', o_orderdate)
    FROM Orders
    ORDER BY strftime('%Y', o_orderdate) DESC
    LIMIT 1;
    """
).fetchall()[0][0]

current_period = int(current_period)

revenue_previous_period = cur.execute(
    """
    SELECT SUM(o_totalprice)
    FROM Orders
    WHERE strftime('%Y', o_orderdate) = ?;
    """,
    (str(current_period - 1),)
    ).fetchall()[0][0]

revenue_this_period = cur.execute(
    """
    SELECT SUM(o_totalprice)
    FROM Orders
    WHERE strftime('%Y', o_orderdate) = ?;
    """,
    (str(current_period),)
    ).fetchall()[0][0]

print(f"Revenues in {current_period}: {round(revenue_this_period, 2):,}")
print(f"Revenues in {current_period - 1}: {round(revenue_previous_period, 2):,}")
print()
print(f"Revenues in 1998 was {revenue_this_period / revenue_previous_period * 100:.2f}% of those in 1997")

Revenues in 1998: 187,332,505.06
Revenues in 1997: 320,228,729.28

Revenues in 1998 was 58.50% of those in 1997


<br>

## Saving changes and closing the database

In [127]:
con.commit()
con.close()